'''
Промежуточная диагностика реализуется в формате решения кейсов. 
Каждый из кейсов опирается на пройденный материал и представляет собой применение нескольких 
упражнений из заданий для самостоятельной работы и практических занятий.


В настоящее время большое распространение получили чат-боты. Они выполняют функции 
автоматизированных сервисов, помогая обрабатывать запросы пользователей. Одним из бурно 
развивающихся направлений представляют собой так называемые умные чат-боты. Это боты, 
общающиеся с пользователями благодаря применению методов обработки естественного языка. 
Искусственный интеллект также может применяться в ботах для обработки изображений. 
Давайте и мы в рамках закрепления пройдённого материала сделаем Telegram-бота, 
которому можно отправлять изображения, а он, используя встроенную предобученную нейросеть, 
будет детектировать объекты на фото. В качестве предобученной нейросети можно 
выбрать YOLO3 (https://pjreddie.com/darknet/yolo/). Бота нужно развернуть на сервере, 
например на платформе Hostman (https://hostman.com/). В качестве демонстрации выполнения 
задания нужно прислать исходный код бота, а также ссылку на него в Telegram, 
чтобы можно было проверить его работоспособность.

Кейс оценивается суммой баллов, получаемых согласно следующим критериям:

Бот успешно выполняет свою функцию — 1 балл;
Программный код бота предоставлен на проверку — 1 балл;
Программный код содержит в себе подключение предобученной нейросети — 1 балл;
Программный код бота содержит в себе обработку передаваемых через сообщения изображений — 1 балл;
Программный код содержит отправку обработанного фото обратно пользователю — 1 балл.
Кейс считается успешно выполненным, если слушатель набрал не менее 3 из 5 возможных баллов.
'''

In [ ]:
'''
!echo "# TelegramBot_mvi" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/MaximIvanovV/TelegramBot_mvi.git
!git push -u origin main
'''

In [3]:
# инсталируем необходимые библиотеки
%pip install pyTelegramBotAPI
%pip install opencv-python


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# импортируем нужные модули

# from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import PIL
import time
import telebot

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [3]:
!git clone https://github.com/AlexeyAB/darknet


fatal: could not create work tree dir 'darknet': Permission denied


In [15]:
# YOLO
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

!make


[Errno 2] No such file or directory: 'darknet'
/
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
make: *** No targets specified and no makefile found.  Stop.


In [ ]:
# скачаем файл весов модели yolov4, натренированный определять 80 классов объектов 
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq" -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

In [ ]:
# импортируем darknet функции для детектирования объектов
from darknet import *
# загрузим YOLOv4 
network, class_names, class_colors = load_network("cfg/yolov4-csp.cfg", "cfg/coco.data", "yolov4-csp.weights")
width = network_width(network)
height = network_height(network)

# darknet_helper функция для детектирования объектов на изображении
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # пересчет координат границ объектов, чтобы можно было отобразить на кадре
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # запуск модели
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

In [ ]:
# функция распознования 
def recognition(img):
  image = cv2.imread(img)
  detections, width_ratio, height_ratio = darknet_helper(image, width, height)

  for label, confidence, bbox in detections:
    left, top, right, bottom = bbox2points(bbox)
    left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
    cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
    cv2.putText(image, "{} [{:.2f}]".format(label, float(confidence)),
                    (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    class_colors[label], 2)
  # cv2_imshow(image)
  return image

In [ ]:
image= recognition('/content/faces.png')

In [ ]:
# Telegram Bot
id= 5489463331
token= '5489463331:AAHZT7WI2de8CTxvH9qEIXh31Z82519UeOA'
bot = telebot.TeleBot(token)
@bot.message_handler(content_types=['photo'])

def object_recon(message):

  file = bot.get_file(message.photo[-1].file_id)
  bot.send_message(message.from_user.id, 'Фото получил. Обрабатываю!')
  dfile = bot.download_file(file.file_path)
  fimg = message.photo[-1].file_id + '.png'
  with open(fimg, 'wb') as new_file:
     new_file.write(dfile)
  new_file.close()

  imgtosend= recognition(fimg)

  cv2.imwrite(fimg, imgtosend)

  with open(fimg,'rb') as fil:
        byte = fil.read()
  print('отправлено. объекты выделены.')
  bot.send_photo(message.from_user.id,  byte)
  cv2_imshow(imgtosend)
  

bot.infinity_polling()

Output hidden; open in https://colab.research.google.com to view.